In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) th
#at gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/__init__.py
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv
/kaggle/input/jpx-tokyo-stock-

# Here starts my Baseline

Ideas:

1  - Feature engeneering- Basic and effective;

2 - Gradient Boosting Regressions;

3 - Neuronets Regressions; and

4 - Submission.



## 1 Feature Engeneering

In [2]:
def concat_df(df1, df2):
    df1 = pd.concat([df1, df2],
                    ignore_index=True, sort=False
                    ).drop_duplicates(["RowId"], keep="first")
    return df1

In [3]:
# For now... use pandas.... 
#If too heavy use Cudf

# For now... just fix the floats
dtypes = {
'SecuritiesCode' :     np.int16,
'Open'           :     np.float16,
'High'           :     np.float16,
'Low'            :     np.float16,
'Close'          :     np.float16,
'Volume'         :       np.int8,
'AdjustmentFactor':    np.float16,
'ExpectedDividend':    np.float16,
'Target'        :      np.float16
}
path = "../input/jpx-tokyo-stock-exchange-prediction/"
df_prices = pd.read_csv(f"{path}train_files/stock_prices.csv",dtype = dtypes)
prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv",dtype = dtypes)
df_prices = concat_df(df_prices, prices)
prices = pd.read_csv(f"{path}train_files/secondary_stock_prices.csv",dtype = dtypes)
df_prices = concat_df(df_prices, prices)
prices = pd.read_csv(f"{path}supplemental_files/secondary_stock_prices.csv",dtype = dtypes)
df_prices = concat_df(df_prices, prices)
#df_prices = df_prices[df_prices.Date<"2021-11-01"]
df_prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5189768 entries, 0 to 5189767
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   RowId             5189768 non-null  object 
 1   Date              5189768 non-null  object 
 2   SecuritiesCode    5189768 non-null  int16  
 3   Open              5080824 non-null  float16
 4   High              5080824 non-null  float16
 5   Low               5080824 non-null  float16
 6   Close             5080824 non-null  float16
 7   Volume            5189768 non-null  int8   
 8   AdjustmentFactor  5189768 non-null  float16
 9   ExpectedDividend  41373 non-null    float16
 10  SupervisionFlag   5189768 non-null  bool   
 11  Target            5188728 non-null  float16
dtypes: bool(1), float16(7), int16(1), int8(1), object(2)
memory usage: 207.9+ MB


In [4]:
# def prep_prices(price):
#     price.fillna(0,inplace=True)
#     return price

In [5]:
# df_prices = prep_prices(df_prices)
# pd.options.display.float_format = '{:,.6g}'.format
# df_prices.describe()

In [6]:
# from datetime import datetime
# import time

#  # auxiliary function, from datetime to timestamp
# totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%Y-%m-%d").timetuple()))

# Data Features


In [7]:

# define function to compute log returns
# def log_return(series, periods=1):
#     return np.log(series).diff(periods=periods)


# # def fill_the_gaps(df):
# #     new_df = pd.DataFrame(columns= df.columns)
# #     for i in range(len(df['SecuritiesCode'].unique())):
# #         new_df = new_df.append(df[df['SecuritiesCode'] == i].reindex(range(df[df['SecuritiesCode'] == i].index[0],df[df['SecuritiesCode'] == i].index[-1]+60,60),method='pad'))
# #     new_df.fillna(method = 'pad', inplace = True) 
# #     return new_df


# def rsiFunc(prices, n=14):
#     deltas = np.diff(prices)
#     seed = deltas[:n+1]
#     up = seed[seed>=0].sum()/n
#     down = -seed[seed<0].sum()/n
#     rs = up/down
#     rsi = np.zeros_like(prices)
#     rsi[:n] = 100. - 100./(1.+rs)

#     for i in range(n, len(prices)):
#         delta = deltas[i-1] # cause the diff is 1 shorter

#         if delta>0:
#             upval = delta
#             downval = 0.
#         else:
#             upval = 0.
#             downval = -delta

#         up = (up*(n-1) + upval)/n
#         down = (down*(n-1) + downval)/n

#         rs = up/down
#         rsi[i] = 100. - 100./(1.+rs)

#     return rsi



def get_features(df):

#     df['log_return_5'] = log_return(df['Close'],periods=5)
#     df['log_return'] = log_return(df['Close'],periods=1)

    upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
    lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    
#     df['EMA_21'] = df['Close'].ewm(span=21).mean()
    
#     df['EMA_55'] = df['Close'].ewm(span=55).mean()
    
#     df['EMA_315'] = df['Close'].ewm(span=315).mean()
    
#     df['EMA_825'] = df['Close'].ewm(span=825).mean()
    
#     window = 7
    
#     no_of_std = 2
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     window = 5
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     df['MACD'] = df['EMA_7'] - df['EMA_5']
    
    
#     df['rsi_5'] = rsiFunc(df['Close'].values, 5)
    
#     df['rsi_7'] = rsiFunc(df['Close'].values, 7)
    
#     df['rsi_21'] = rsiFunc(df['Close'].values, 21)
    
    
    
    df['VWAP'] = (df['Close'] * df['Volume'])/ df['Volume']




    
    
    #df = pd.concat([df, pd.get_dummies(df['Asset_ID'], prefix= 'Asset_')], axis=1)
    
    df[[ 'upper_shadow', 'lower_shadow','VWAP']].astype('float16')
    
    return df



In [8]:
#sort the database per stock before running the feature function
#df_prices = df_prices.sort_values('SecuritiesCode')

df_prices = get_features(df_prices)

df_prices = df_prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

df_prices = df_prices.fillna(0)

## Training a Simple Model

GB

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
def model_training(X,y):
    # Model training
    
    model = GradientBoostingRegressor() 
    model.fit(X, y)
    
    return model


In [11]:
scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']
features = ["SecuritiesCode",'Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']

In [12]:
# Creating endog and exog variables
y = df_prices[['Target']]
X = df_prices[features]

In [13]:
X[scaled_features] = scaler.fit_transform(X[scaled_features])


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
%%time

model_1 = model_training(X,y)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 22min 57s, sys: 9.2 s, total: 23min 6s
Wall time: 23min 6s


In [15]:
model_1.score(X,y)

0.0024390177891785436

In [16]:
import pickle

In [17]:
# Saving model

#Saving the sklearn model

filename = 'simple_skgb.sav'
pickle.dump(model_1, open(filename, 'wb'))

# Submission


In [18]:


def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [19]:
import jpx_tokyo_market_prediction


env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = get_features(prices)
    prices = prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
    prices = prices.fillna(0)
    prices[scaled_features] = scaler.fit_transform(prices[scaled_features])
    prices['pred'] = model_1.predict(prices[features])
    prices = add_rank(prices)
    feature_map = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
